<a href="https://colab.research.google.com/github/Flymshady/GA14_server_tests/blob/main/traffic_flask_pytorch_GA14_%2B_testy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalace & importy


In [1]:
! pip install -q pytorch-lightning
! pip install -q pytorch-lightning-bolts
!pip install flask-ngrok

     |████████████████████████████████| 675kB 8.8MB/s 
     |████████████████████████████████| 829kB 19.0MB/s 
     |████████████████████████████████| 276kB 16.0MB/s 
     |████████████████████████████████| 102kB 11.7MB/s 
     |████████████████████████████████| 194kB 9.4MB/s 


In [2]:
import os
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import transforms, datasets
import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
import torchvision
import numpy as np
import matplotlib.pyplot as plt

## Model


In [3]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [4]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
fid = drive.ListFile({'q':"title='entire_model_GA14_2.pt'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('entire_model_GA14_2.pt')

In [6]:
class_names = ['error', 'no_traffic', 'traffic']

In [7]:
class TrafficModel(pl.LightningModule):

    def __init__(self, num_target_classes):
        super().__init__()
        self.model = torchvision.models.googlenet(pretrained=True) #vyuziti preneseneho uceni z modelu resnet50
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, num_target_classes)
        self.acc = pl.metrics.Accuracy()

    def forward(self, x):
      return self.model(x)

    def training_step(self, batch, batch_idx):  
       
       x, y = batch #rozdeleni batche na jeho dva prvky - vstup a cile(labely)

       y_hat = self(x)  #output modelu

       loss = F.cross_entropy(y_hat, y) #vypocet chyby - ztratova funkce cross_entropy
       self.log('train_loss', loss)   #ulozeni ztraty do logu
       self.log('train_acc_step', self.acc(y_hat, y), on_step=True, on_epoch=False)  #y_hat - predikovana, y - co melo byt predikovanp -> vypocet kolik % z toho bylo spravne
       return loss

    def training_epoch_end(self, outs):
      acc = self.acc.compute()
      self.log("train_acc_epoch", acc)

    def validation_step(self, batch, batch_idx):

        x, y = batch
        y_hat = self(x)  #output modelu
        loss = F.cross_entropy(y_hat, y) #vypocet chyby - ztratova funkce cross_entropy
        self.log('val_loss', loss)
  
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

model = TrafficModel(len(class_names))

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


In [8]:
model = torch.load('entire_model_GA14_2.pt')
model.eval()

TrafficModel(
  (model): GoogLeNet(
    (conv1): BasicConv2d(
      (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (conv2): BasicConv2d(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv3): BasicConv2d(
      (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (inception3a): Inception(
      (branch1): BasicConv2d(
        (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn)

## Příprava & zpracování obrázku


In [9]:
import os

# define the name of the directory to be created
os.chdir('/content/')
path = "/content/actuall"

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)

os.chdir('/content')

Successfully created the directory /content/actuall 


In [10]:
import requests
def download_image(place_t):
  os.chdir('/content/actuall')
  with open(place_t+'.jpg', 'wb') as handle:
        response = requests.get('http://kamery.lwq.cz/jpg/'+place_t+'.jpg', stream=True)
        if not response.ok:
            return 'Chyba'
        for block in response.iter_content(1024):
            if not block:
                break
            handle.write(block)

In [27]:
from PIL import Image

#device = torch.device("cuda")

def classify_traffic_place(model, place):
  place=int(place)
  if place == 0:
    place_t = 'barton-vysokov'
  elif place == 1:
    place_t = 'barton-centrum'
  elif place == 2:
    place_t = 'barton'
  elif place == 3:
    place_t = 'slavia'
  elif place == 4:
    place_t = 'italie'
  elif place == 5:
    place_t = 'polska'
  else: return 'Chyba'
  os.chdir('/content/actuall')
  """
  if os.path.exists('/content/actuall/'+place_t+'.jpg'):
      os.remove('/content/actuall/'+place_t+'.jpg')
 
  download_image(place_t)
 
  
  img = Image.open('/content/actuall/'+place_t+'.jpg')
"""
  img=Image.open('/content/actuall/test.jpg')

  transform_image = transforms.Compose([
                                 transforms.Resize((224,224)),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

  img_t = transform_image(img)
  batch_t = torch.unsqueeze(img_t, 0)


  #device = torch.device("cuda")
  model.eval()
  #batch_t = batch_t.to(device)
  out = model(batch_t)
  print("--------out------")
  print(out)

  _, classifs = torch.max(out, 1)
  print(_, classifs)
  return class_names[classifs]
  #model.to(device)

## Flask & Ngrok


In [19]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run.
@app.route('/predict/<id>/', methods=['POST', 'GET'])
def predict(id):

    return jsonify([{"pred": classify_traffic_place(model, id)}])
 
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://58ec9966a294.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


## Jiné

In [53]:
#vlastní tesovací metoda

id=1
print(classify_traffic_place(model, id))


--------out------
tensor([[-17.5986,  -1.3307,   1.5738]], grad_fn=<AddmmBackward>)
tensor([1.5738], grad_fn=<MaxBackward0>) tensor([2])
traffic


## Testy


In [ ]:
fid = drive.ListFile({'q':"title='testdata.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('testdata.zip')

In [ ]:
import zipfile

with zipfile.ZipFile("testdata.zip", 'r') as zip_ref:  #with pro otevreni i zavreni souboru
  zip_ref.extractall()

No_traffic test


In [62]:
for filename in os.listdir('testdata/no_traffic'): 
 
  img=Image.open('/content/testdata/no_traffic/'+filename)

  transform_image = transforms.Compose([
                                 transforms.Resize((224,224)),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

  img_t = transform_image(img)
  batch_t = torch.unsqueeze(img_t, 0)


  #device = torch.device("cuda")
  model.eval()
  #batch_t = batch_t.to(device)
  out = model(batch_t)
  _, classifs = torch.max(out, 1)
  #print(class_names[classifs])
  if class_names[classifs] != "no_traffic":
    print(filename)
  #model.to(device)

bar_d_c_14_12_11_27_47.jpg
bar_d_c_14_12_11_17_39.jpg
bar_d_c_14_12_11_22_43.jpg
bar_d_c_14_12_11_12_35.jpg


Traffic test

In [ ]:
for filename in os.listdir('testdata/traffic'): 
 
  img=Image.open('/content/testdata/traffic/'+filename)

  transform_image = transforms.Compose([
                                 transforms.Resize((224,224)),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

  img_t = transform_image(img)
  batch_t = torch.unsqueeze(img_t, 0)


  #device = torch.device("cuda")
  model.eval()
  #batch_t = batch_t.to(device)
  out = model(batch_t)
  _, classifs = torch.max(out, 1)
  #print(class_names[classifs])
  if class_names[classifs] != "traffic":
    print(filename)
  #model.to(device)

Error test

In [ ]:
for filename in os.listdir('testdata/error'): 
 
  img=Image.open('/content/testdata/error/'+filename)

  transform_image = transforms.Compose([
                                 transforms.Resize((224,224)),
                                 transforms.ToTensor(),
                                 transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])

  img_t = transform_image(img)
  batch_t = torch.unsqueeze(img_t, 0)


  #device = torch.device("cuda")
  model.eval()
  #batch_t = batch_t.to(device)
  out = model(batch_t)
  _, classifs = torch.max(out, 1)
  #print(class_names[classifs])
  if class_names[classifs] != "error":
    print(filename)
  #model.to(device)